
# Market basket analysis at the grocery outlet

### Introduction

**Market basket analysis** tells us which products tend to be purchased together and which are most amenable to promotion. This information is actionable: it can suggest new store layouts, determine which articles to put on special, indicate when to issue coupons, and so on. When these data can be tied to individual customers through a loyalty card or website registration, they become even more valuable. The application of **association rules** to market basket analysis is a classic of data mining. 

In this example, a Chicago-based marketing analyst focusing on the retail industry explores different approaches for modeling consumer behavior using data on **point-of-sale transactions** in small stores of the Chicago metropolitan area. She starts with a market basket analysis of data from a typical local grocery outlet, where she intends to identify **joint occurrence** of products in shopping baskets.

### The data set

The `groceries` data set covers one month of point-of-sale **transaction data**. It contains 9,835 transactions and the items are aggregated to 169 categories. The data come as a **matrix transaction/item**: an entry equal to 1 in the intersection of row *i* and column column *j* indicates that transaction *i* includes item *j*. Note the **sparsity** of the data: there are only 43,367 nonzero entries, out of the 1,662,115 terms of this matrix (2.6%). So, this is an inefficient way of transporting the data, even if it can be used in this example to keep it simple.

### Mining itemsets

The analysis of this example is based on the function `apriori` of the R package `arules`. It is described as two steps: (a) extracting the most frequent itemsets, and (b) selecting association rules by support and confidence. This is done here for pedagogical purposes, although we do not always find these two steps separated in data mining software.

I start by loading the package and the data.

In [1]:
library(arules)

Loading required package: Matrix

Attaching package: ‘arules’

The following objects are masked from ‘package:base’:

    abbreviate, write



In [2]:
groceries = read.csv("https://raw.githubusercontent.com/iese-bad/DataSci/master/Data/groceries.csv")

Actually, `groceries` is a data frame, with 9,385 rows and 169 columns, as shown by

In [3]:
print(dim(groceries))

[1] 9835  169


The package `arules` deals with many data formats, including two proprietary ones, `transactions`, and `tidLists`. To keep it simple, I ignore these formats, coercing the data frame `groceries` into a (binary) matrix, which
is also accepted by `apriori`.

In [4]:
groceries = as.matrix(groceries)

I start with the itemsets of size 1, filtering out the items that appear in less than 1% (trial-and-error approach) of the transactions. I specify this with the parameters `support`, `target`, `maxlen` and `minlen`. Do not pay much attention to the output that R is returning, except for the information that we have captured 88 items.

In [5]:
item1 = apriori(groceries, parameter=list(support=0.01, minlen=1, maxlen=1,
    target='frequent itemsets'))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
         NA    0.1    1 none FALSE            TRUE       5    0.01      1
 maxlen            target   ext
      1 frequent itemsets FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 98 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[169 item(s), 9835 transaction(s)] done [0.00s].
sorting and recoding items ... [88 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1

Warning message in apriori(groceries, parameter = list(support = 0.01, minlen = 1, :
“Mining stopped (maxlen reached). Only patterns up to a length of 1 returned!”

 done [0.01s].
writing ... [88 set(s)] done [0.00s].
creating S4 object  ... done [0.00s].


I sort the items by support, picking the first 20. Here, `item1` is an object of the class `itemsets`. You do not need to understand what this really means. It is enough to look at the content, that is, to list the items, which can be done with the function `inspect`.

In [6]:
item1 = sort(item1, by='support')[1:20]
inspect(item1)

     items                   support   
[1]  {whole_milk}            0.25551601
[2]  {other_vegetables}      0.19349263
[3]  {rolls_buns}            0.18393493
[4]  {soda}                  0.17437722
[5]  {yogurt}                0.13950178
[6]  {bottled_water}         0.11052364
[7]  {root_vegetables}       0.10899847
[8]  {tropical_fruit}        0.10493137
[9]  {shopping_bags}         0.09852567
[10] {sausage}               0.09395018
[11] {pastry}                0.08896797
[12] {citrus_fruit}          0.08276563
[13] {bottled_beer}          0.08052872
[14] {newspapers}            0.07981698
[15] {canned_beer}           0.07768175
[16] {pip_fruit}             0.07564820
[17] {fruit_vegetable_juice} 0.07229283
[18] {whipped_sour_cream}    0.07168277
[19] {brown_bread}           0.06487036
[20] {domestic_eggs}         0.06344687


For the two-item itemsets, I set `maxlen` and `minlen` to 2, capturing 213 itemsets.

In [7]:
item2 = apriori(groceries, parameter=list(support=0.01, minlen=2, maxlen=2,
    target='frequent itemsets'))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
         NA    0.1    1 none FALSE            TRUE       5    0.01      2
 maxlen            target   ext
      2 frequent itemsets FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 98 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[169 item(s), 9835 transaction(s)] done [0.00s].
sorting and recoding items ... [88 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2

Warning message in apriori(groceries, parameter = list(support = 0.01, minlen = 2, :
“Mining stopped (maxlen reached). Only patterns up to a length of 2 returned!”

 done [0.01s].
writing ... [213 set(s)] done [0.00s].
creating S4 object  ... done [0.00s].


Now, we apply `inspect` as we did above. We see in the report that the minimum support must be set at 0.03 to get at least 20 two-item itemsets (we need at least
two items for a rule). 

In [8]:
item2 = sort(item2, by='support')[1:20]
inspect(item2)

     items                              support   
[1]  {other_vegetables,whole_milk}      0.07483477
[2]  {whole_milk,rolls_buns}            0.05663447
[3]  {whole_milk,yogurt}                0.05602440
[4]  {root_vegetables,whole_milk}       0.04890696
[5]  {root_vegetables,other_vegetables} 0.04738180
[6]  {other_vegetables,yogurt}          0.04341637
[7]  {other_vegetables,rolls_buns}      0.04260295
[8]  {tropical_fruit,whole_milk}        0.04229792
[9]  {whole_milk,soda}                  0.04006101
[10] {rolls_buns,soda}                  0.03833249
[11] {tropical_fruit,other_vegetables}  0.03589222
[12] {whole_milk,bottled_water}         0.03436706
[13] {yogurt,rolls_buns}                0.03436706
[14] {whole_milk,pastry}                0.03324860
[15] {other_vegetables,soda}            0.03274021
[16] {whole_milk,whipped_sour_cream}    0.03223183
[17] {sausage,rolls_buns}               0.03060498
[18] {citrus_fruit,whole_milk}          0.03050330
[19] {pip_fruit,whole_milk}    

For the 3-item itemsets, I set `maxlen` and `minlen` to 3, capturing 32 itemsets.

In [9]:
item3 = apriori(groceries, parameter=list(support=0.01, minlen=3, maxlen=3,
    target='frequent itemsets'))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
         NA    0.1    1 none FALSE            TRUE       5    0.01      3
 maxlen            target   ext
      3 frequent itemsets FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 98 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[169 item(s), 9835 transaction(s)] done [0.00s].
sorting and recoding items ... [88 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 3

Warning message in apriori(groceries, parameter = list(support = 0.01, minlen = 3, :
“Mining stopped (maxlen reached). Only patterns up to a length of 3 returned!”

 done [0.01s].
writing ... [32 set(s)] done [0.00s].
creating S4 object  ... done [0.00s].


I use again `inspect` to list the most frequent itemsets. Note that support is going down.

In [10]:
item3 = sort(item3, by='support')[1:20]
inspect(item3)

     items                                             support   
[1]  {root_vegetables,other_vegetables,whole_milk}     0.02318251
[2]  {other_vegetables,whole_milk,yogurt}              0.02226741
[3]  {other_vegetables,whole_milk,rolls_buns}          0.01789527
[4]  {tropical_fruit,other_vegetables,whole_milk}      0.01708185
[5]  {whole_milk,yogurt,rolls_buns}                    0.01555669
[6]  {tropical_fruit,whole_milk,yogurt}                0.01514997
[7]  {other_vegetables,whole_milk,whipped_sour_cream}  0.01464159
[8]  {root_vegetables,whole_milk,yogurt}               0.01453991
[9]  {other_vegetables,whole_milk,soda}                0.01392984
[10] {pip_fruit,other_vegetables,whole_milk}           0.01352313
[11] {citrus_fruit,other_vegetables,whole_milk}        0.01301474
[12] {root_vegetables,other_vegetables,yogurt}         0.01291307
[13] {root_vegetables,whole_milk,rolls_buns}           0.01270971
[14] {other_vegetables,whole_milk,domestic_eggs}       0.01230300
[15] {trop

### Mining rules

Finally, I go for the rules. I set `target='rules'` and `confidence=0.25` (the default is 0.8, too high). Note that, here, instead of a minimum support, I am setting a minimum confidence, which makes more sense for rules. For the rules with one-item antecedent, I set `maxlen` and `minlen` to 2, capturing 96 rules. 

In [11]:
rules = apriori(groceries, parameter=list(support=0.01, confidence = 0.25, maxlen=2, minlen=2,
    target='rules'))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
       0.25    0.1    1 none FALSE            TRUE       5    0.01      2
 maxlen target   ext
      2  rules FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 98 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[169 item(s), 9835 transaction(s)] done [0.01s].
sorting and recoding items ... [88 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2

Warning message in apriori(groceries, parameter = list(support = 0.01, confidence = 0.25, :
“Mining stopped (maxlen reached). Only patterns up to a length of 2 returned!”

 done [0.01s].
writing ... [96 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].


I sort the rules, now by confidence. `rules` is an object of the class `rules` (I don't discuss this). I use `inspect` to take a look at it.

In [12]:
rules = sort(rules, by='confidence')[1:20]
inspect(rules)

     lhs                     rhs                support    confidence lift    
[1]  {butter}             => {whole_milk}       0.02755465 0.4972477  1.946053
[2]  {curd}               => {whole_milk}       0.02613116 0.4904580  1.919481
[3]  {domestic_eggs}      => {whole_milk}       0.02999492 0.4727564  1.850203
[4]  {onions}             => {other_vegetables} 0.01423488 0.4590164  2.372268
[5]  {whipped_sour_cream} => {whole_milk}       0.03223183 0.4496454  1.759754
[6]  {root_vegetables}    => {whole_milk}       0.04890696 0.4486940  1.756031
[7]  {sugar}              => {whole_milk}       0.01504830 0.4444444  1.739400
[8]  {hamburger_meat}     => {whole_milk}       0.01474326 0.4434251  1.735410
[9]  {ham}                => {whole_milk}       0.01148958 0.4414062  1.727509
[10] {sliced_cheese}      => {whole_milk}       0.01077783 0.4398340  1.721356
[11] {root_vegetables}    => {other_vegetables} 0.04738180 0.4347015  2.246605
[12] {frozen_vegetables}  => {whole_milk}       0.02

### Source of the data

M Haessler, K Hornik & T Reutterer (2006), Implications of probabilistic data modeling for mining association rules, in *From Data and Information Analysis to Knowledge Engineering*, Springer, pages 598-605.